In [1]:
import numpy as np
import sympy

i^j=0
i^j=1
i^j=1
i^j=0


In [185]:
class Wire:
    def value(self):
        raise NotImplemented

class InputWire(Wire):
    def __init__(self, val, symbol=None):
        assert val == 0 or val == 1 or val is None
        self._val = val
        self.symbol = symbol

    def value(self):
        return self._val

class OutputWire(Wire):
    def __init__(self, func, symbol=None):
        self.symbol = symbol
        self._func = func

    def value(self):
        return self._func()

class Gate:
    def __init__(self, lwire: Wire, rwire: Wire, symbol=None):
        self.symbol = symbol
        self._lwire = lwire
        self._rwire = rwire
        self._outwire = None

    def output(self) -> Wire:
        if self._outwire is not None:
            return self._outwire

        self._outwire = self._get_outwire()
        return self._outwire

    def _get_outwire(self) -> Wire:
        raise NotImplementedError

class XorGate(Gate):
    def _get_outwire(self):
        return OutputWire(
            lambda: self._lwire.value() ^ self._rwire.value(),
            symbol=self.symbol
        )

    def _linearization_coeffs(self):
        # a + b + c \in {0,2}
        return 1, 1, 1

    def _linearization_bias(self):
        return 0

class OrGate(Gate):
    def _get_outwire(self):
        return OutputWire(
            lambda: self._lwire.value() | self._rwire.value(),
            symbol=self.symbol
        )
    def _linearization_coeffs(self):
        #      !a + !b - 2!c \in {0,1}
        # <=>  1 -a  + 1 -b -2 + 2c \in {0,1}
        # <=>  -a -b +2c +0 \in {0,1}
        # <=> -2a -2b +4c \in {0,2}
        return -2, -2, 4

    def _linearization_bias(self):
        # see coeffs
        return 0

class AndGate(Gate):
    def _get_outwire(self):
        return OutputWire(
            lambda: self._lwire.value() & self._rwire.value(),
            symbol=self.symbol
        )

    def _linearization_coeffs(self):
        #     a + b - 2c \in {0,1}
        # <=> 2a + 2b - 4c \in {0,2}
        return 2, 2, -4

    def _linearization_bias(self):
        return 0

class Circuit:
    def __init__(self, inputs: list[InputWire], symbol="C"):
        self._wires = [i for i in inputs] # hoping this copies?
        self._gates = []
        self._output_gate = None
        self.symbol = symbol

    def add_gate(self, gate: Gate):
        # TODO: add plausibility check
        assert gate._lwire in self._wires
        assert gate._rwire in self._wires
        
        self._gates.append(gate)
        self._wires.append(gate.output())

    def set_outgate(self, gate: Gate):
        assert gate in self._gates
        self._output_gate = gate

    def eval(self):
        if self._output_gate is None:
            print("no output gate set, using last one")
            return self._gates[-1].output().value()
        return self._output_gate.output().value()

    def size(self) -> (int, int):
        # m x n
        # wires x gates
        return len(self._wires), len(self._gates)

    def _gate_wire_idxs(self, gate):
        """return the indices of the gate's wires. -> left input, right input, output"""
        assert gate in self._gates
        l = self._wires.index(gate._lwire)
        r = self._wires.index(gate._rwire)
        o = self._wires.index(gate.output())

        return l, r, o

    def matrix_G(self):
        assert self._output_gate is not None, "need output gate"
        
        G = np.zeros(self.size())
        
        for i, gate in enumerate(self._gates):
            li, ri, oi = self._gate_wire_idxs(gate)
            lc, rc, oc = gate._linearization_coeffs()
            if gate == self._output_gate:
                oc -= 3
            G[li, i] = lc
            G[ri, i] = rc
            G[oi, i] = oc

        return G

    def vector_delta(self):
        assert self._output_gate is not None, "need output gate"

        _, n = self.size()
        delta = np.zeros(n)
        
        for i, gate in enumerate(self._gates):
            b = gate._linearization_bias()
            if gate == self._output_gate:
                b += 3
            delta[i] = b

        return delta

    def matrix_V(self):
        m, n = self.size()
        return np.concat([2*np.eye(m), self.matrix_G()], axis=1)

    def vector_b(self):
        m, _ = self.size()
        return np.concat([np.zeros(m), self.vector_delta()])
            
            
        

In [118]:
np.concat([np.eye(3), np.zeros((3,2))])

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 3 and the array at index 1 has size 2

In [47]:
for i in [0,1]:
    for j in [0, 1]:
        print(f"{i=},{j=}")
        print(f"{AndGate(InputWire(i), InputWire(j)).output().value()=}")
        print(f"{OrGate(InputWire(i), InputWire(j)).output().value()=}")
        print(f"{XorGate(InputWire(i), InputWire(j)).output().value()=}")
        print()


i=0,j=0
AndGate(InputWire(i), InputWire(j)).output().value()=0
OrGate(InputWire(i), InputWire(j)).output().value()=0
XorGate(InputWire(i), InputWire(j)).output().value()=0

i=0,j=1
AndGate(InputWire(i), InputWire(j)).output().value()=0
OrGate(InputWire(i), InputWire(j)).output().value()=1
XorGate(InputWire(i), InputWire(j)).output().value()=1

i=1,j=0
AndGate(InputWire(i), InputWire(j)).output().value()=0
OrGate(InputWire(i), InputWire(j)).output().value()=1
XorGate(InputWire(i), InputWire(j)).output().value()=1

i=1,j=1
AndGate(InputWire(i), InputWire(j)).output().value()=1
OrGate(InputWire(i), InputWire(j)).output().value()=1
XorGate(InputWire(i), InputWire(j)).output().value()=0



In [123]:
a1 = InputWire(0, symbol="a")
b1 = InputWire(1, symbol="b")

C1 = Circuit([a1,b1], symbol="C")
g = XorGate(a1,b1, symbol="c")
C1.add_gate(g)
C1.set_outgate(g)
print(C1.size())
C1._wires

(3, 1)


In [124]:
a = InputWire(0, symbol="a")
b = InputWire(1, symbol="b")
c = InputWire(1, symbol="c")
d = InputWire(1, symbol="d")

C = Circuit([a,b, c, d], symbol="C")
aorb = OrGate(a,b, symbol="a|b")
C.add_gate(aorb)
candd = AndGate(c, d, symbol="c&d")
C.add_gate(candd)

final_gate = XorGate(aorb.output(), candd.output(), symbol="o")
C.add_gate(final_gate)
C.set_outwire(final_gate.output())
C._wires

AttributeError: 'Circuit' object has no attribute 'set_outwire'

In [125]:
C1.eval()

1

In [126]:
C1.matrix_G(), C1.vector_delta()

(array([[ 1.],
        [ 1.],
        [-2.]]),
 array([3.]))

In [130]:
V1 = C1.matrix_V()
b1 = C1.vector_b()

V1, b1

(array([[ 2.,  0.,  0.,  1.],
        [ 0.,  2.,  0.,  1.],
        [ 0.,  0.,  2., -2.]]),
 array([0., 0., 0., 3.]))

In [159]:
a = np.array([0,0,0])

In [160]:
a@V1 + b1

array([0., 0., 0., 3.])

In [151]:
bla = a @ V1 + b1
bool(np.logical_or(bla == 0, bla ==2).all())

False

In [189]:
def accept(a, V, b):
    a = np.array(a)
    out = a @ V + b
    test = np.logical_or(out == 2, out == 0)
    return bool(test.all())

def test_xor():
    a = InputWire(None, symbol="a")
    b = InputWire(None, symbol="b")

    C = Circuit([a, b], symbol="C")
    g = XorGate(a,b, symbol="c")
    C.add_gate(g)
    C.set_outgate(g)

    V, b = C.matrix_V(), C.vector_b()
    for l in [0,1]:
        for r in [0,1]:
            if l^r:
                assert accept([l, r, 1], V, b)
            else:
                assert not accept([l, r, 0], V, b)
                assert not accept([l, r, 1], V, b)

    print("xor passed")

def test_or():
    a = InputWire(None, symbol="a")
    b = InputWire(None, symbol="b")

    C = Circuit([a, b], symbol="C")
    g = OrGate(a,b, symbol="c")
    C.add_gate(g)
    C.set_outgate(g)

    V, b = C.matrix_V(), C.vector_b()
    for l in [0,1]:
        for r in [0,1]:
            if l or r:
                assert accept([l, r, 1], V, b), [l, r, 1]
            else:
                assert not accept([l, r, 0], V, b), [l, r, 0]
                assert not accept([l, r, 1], V, b), [l, r, 1]

    print("or passed")

def test_and():
    a = InputWire(None, symbol="a")
    b = InputWire(None, symbol="b")

    C = Circuit([a, b], symbol="C")
    g = AndGate(a,b, symbol="c")
    C.add_gate(g)
    C.set_outgate(g)

    V, b = C.matrix_V(), C.vector_b()
    for l in [0,1]:
        for r in [0,1]:
            if l and r:
                assert accept([l, r, 1], V, b), [l, r, 1]
            else:
                assert not accept([l, r, 0], V, b), [l, r, 0]
                assert not accept([l, r, 1], V, b), [l, r, 1]

    print("and passed")

In [190]:
test_xor()
test_and()
test_or()

xor passed
and passed
or passed
